<a href="https://www.kaggle.com/code/vanshikaajindal/bert-lda?scriptVersionId=134489440" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import shutil
!pip install tensorflow
import tensorflow as tf
!pip install tensorflow_text

import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

data_folder = '/kaggle/input/text-summarization/data'

# Load train data
train_data = []
train_labels = []
class_names = []

train_folder = os.path.join(data_folder, 'train')
for class_folder in os.listdir(train_folder):
    if os.path.isdir(os.path.join(train_folder, class_folder)):
        class_names.append(class_folder)
        class_path = os.path.join(train_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    train_data.append(file.read())
                    train_labels.append(class_folder)

# Load test data
test_data = []
test_labels = []

test_folder = os.path.join(data_folder, 'test')
for class_folder in os.listdir(test_folder):
    if os.path.isdir(os.path.join(test_folder, class_folder)):
        class_path = os.path.join(test_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    test_data.append(file.read())
                    test_labels.append(class_folder)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, precision_score
from sklearn.metrics import confusion_matrix

# Define BERT model
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_model_path = '/kaggle/input/bert-model-1'  # Replace with the actual file path
bert_preprocess_model_path = '/kaggle/input/preocessor'  # Replace with the actual file path

# Load BERT model and preprocessing model
bert_preprocess_model = hub.KerasLayer(bert_preprocess_model_path)
bert_model = hub.KerasLayer(bert_model_path)

# Preprocess the data using BERT
train_preprocessed = bert_preprocess_model(train_data)
train_embeddings = bert_model(train_preprocessed)['pooled_output']

# Convert BERT embeddings to document-term matrix using CountVectorizer
vectorizer = CountVectorizer()
train_doc_term_matrix = vectorizer.fit_transform(train_data)


In [3]:
# Apply LDA for dimensionality reduction
lda = LatentDirichletAllocation(n_components=10)  # Adjust the number of components as needed
train_lda_features = lda.fit_transform(train_doc_term_matrix)

# Convert labels to numeric format
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Train a classifier on LDA features
classifier = Pipeline([
    ('classifier', tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ]))
])

classifier['classifier'].compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(train_lda_features, train_labels_encoded)

# Preprocess and classify test data
test_preprocessed = bert_preprocess_model(test_data)
test_embeddings = bert_model(test_preprocessed)['pooled_output']
test_doc_term_matrix = vectorizer.transform(test_data)
test_lda_features = lda.transform(test_doc_term_matrix)
predictions = classifier.predict(test_lda_features)



63/63 [==============================] - 0s 1ms/step


In [4]:
# Convert predictions back to labels
predicted_labels_encoded = tf.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded.numpy())
# Convert labels to numeric format
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Create and fit the OneHotEncoder
one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(train_labels_encoded.reshape(-1, 1))

# Convert labels to one-hot encoded format
train_labels_one_hot = one_hot_encoder.transform(train_labels_encoded.reshape(-1, 1)).astype(float)
test_labels_one_hot = one_hot_encoder.transform(test_labels_encoded.reshape(-1, 1)).astype(float)

# Train a classifier on LDA features
classifier = Pipeline([
    ('classifier', tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ]))
])

classifier['classifier'].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(train_lda_features, tf.keras.utils.to_categorical(train_labels_encoded))

# Preprocess and classify test data
test_preprocessed = bert_preprocess_model(test_data)
test_embeddings = bert_model(test_preprocessed)['pooled_output']
test_doc_term_matrix = vectorizer.transform(test_data)
test_lda_features = lda.transform(test_doc_term_matrix)
predictions = classifier.predict(test_lda_features)

# Convert predictions back to labels
predicted_labels_encoded = tf.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded.numpy())

# Convert labels to numeric format
test_labels_encoded = label_encoder.transform(test_labels)

# Evaluate the classifier
accuracy = tf.metrics.Accuracy()
accuracy.update_state(test_labels_encoded, predicted_labels_encoded)
loss = classifier['classifier'].evaluate(test_lda_features, tf.keras.utils.to_categorical(test_labels_encoded), verbose=0)[0]
f1 = f1_score(test_labels, predicted_labels, average='weighted')
precision = precision_score(test_labels, predicted_labels, average='weighted')
confusion = confusion_matrix(test_labels, predicted_labels)

print('Accuracy:', accuracy.result().numpy())
print('Loss:', loss)
print('F1 Score:', f1)
print('Precision:', precision)
print(confusion)

63/63 [==============================] - 0s 2ms/step
Accuracy: 0.5919081
Loss: 1.0699849128723145
F1 Score: 0.512390946957899
Precision: 0.4996890216552204
[[  0  56  19 155   0]
 [  0 189  11 277   0]
 [  0  14  54 189   0]
 [  0  20  20 942   0]
 [  0  43   0  13   0]]


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
